This kernel is using pre-train model to be the base model, and add some extra layers for our training model. <br />
Since the training dataset only has 4000 pictures, so it might be not easy to train a model with excellent feature extracting ability on this size of dataset. <br />
Therefore, transfer-learning is our good friend on this task, since it had pre-trained to have quite nice feature extracting ability on imagenet dataset. <br />
And it is pretty easy to get over 99% accuracy by using technique. <br />
But I still suggest to build our own model for this task. After all, we are here for learning.

In [ ]:
import numpy as np 
import pandas as pd 
import os
train_dir = "../input/data/kaggle_dogcat/train/"
test_dir =  "../input/data/kaggle_dogcat/test/"

# Load all the filenames (Actually we can use "flow_from_directory" function
# from ImageDataGenerator to load the image, in case the momory is tight)
train_dog_fns = os.listdir(train_dir + "dogs/")
train_cat_fns = os.listdir(train_dir + "cats/")
test_fns = os.listdir(test_dir)

#############################################################
#############################################################
#Dont forget to sort the file name for matching submission ID
test_fns.sort()
#############################################################
############################################################

# Check the legnth of all data first
print(f"dog images in training set : {len(train_dog_fns)}")
print(f"cat images in training set : {len(train_cat_fns)}")
print(f"total images in test set : {len(test_fns)}")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, Sequential
from pickle import dump
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization, AveragePooling2D
from tensorflow.keras.optimizers import Adam
import keras
import keras.backend as K
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
base_model = ResNet50(include_top = False, input_shape = (224,224,3), input_tensor = None)
base_model.summary()

In [ ]:
#connect the output of our base model to new layers we created
x = base_model.output
x = AveragePooling2D(pool_size = (7,7))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(units = 1024)(x)
x = BatchNormalization(axis = 1)(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
x = Dense(units = 512)(x)
x = BatchNormalization(axis = 1)(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
output_layer = Dense(units = 2, activation = "softmax")(x)

model = Model(inputs = base_model.input, outputs = output_layer)
optimizer = Adam(lr = 1e-4, decay = 1e-6)
model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ['accuracy'])
model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.utils import shuffle


#This two array is for channel-base mean subtraction preprocessing
#although my score had became worse after using it. 
#pre-calculated mean value of each channel of training  dataset
#Pre-calculate std values of each channel of training dataset
train_mean = np.array([0.4914, 0.458, 0.4193], dtype = 'float64')
train_std = np.array([0.2591,0.2524,0.25511] , dtype = 'float64')

def load_target_img(grayscale = False, color_mode = 'rgb'
                    , target_size = (224,224,3), interpolation = 'bicubic'):
    '''
    grayscale : loading image with color type or grayscale type
    color_mode : loading image with rgb color space and the channels are (r,g,b)
    target_size :Image will be rescale/resize to the target size when loading
    interpolation : the interpolation algorithm for rescaling/resizing the image, nearest is the most simplest and fastest one 
    '''
    def f(path):
        return load_img(path = path, grayscale = grayscale, color_mode = color_mode,
                              target_size = target_size, interpolation=interpolation)
    return f

# I created an image generator to generate the image in batch size
# In case the momery is not enough

def image_generator(load_dir, dog_fns, cat_fns, batch_size = 32):
    '''
    load_dir : data loading path
    dog_fns : filenames of dog picture
    cat_fns : filenames of cat picture
    batch_size : how many data we package as a batch
    '''
    dog_load_dir = load_dir + "dogs/"
    cat_load_dir = load_dir + "cats/"
    data_batch = []
    label_batch = []
    
    while True:
        for idx in range(min(len(dog_fns),len(cat_fns))):
            
            # We should check the file is .jpg format or not
            if ".jpg" in dog_fns[idx] and ".jpg" in cat_fns[idx]:
                data_batch.append( np.asarray(load_target_img()(dog_load_dir + dog_fns[idx])) ) 
                label_batch.append([1,0]) # setting dog label to 0
                data_batch.append( np.asarray(load_target_img()(cat_load_dir + cat_fns[idx])) )
                label_batch.append([0,1]) # setting cat label to 1
                    
                # When the batch collecting is reach to batch_size
                # The data is ready to shipping out
                # yield can keep the status of your function 
                if len(data_batch) >= batch_size :
                    yield np.stack(data_batch), np.stack(label_batch)
                    data_batch = []
                    label_batch = []
                
def image_augmentation_generator(input_gen, image_gen):
    '''
    input_gen : input image generator, it will generate a batch of data a time
    image_gen : ImageDataGenerator from keras
    '''
    for data, label in input_gen:
        
        data,label = shuffle(data, label, random_state = np.random.choice(2019,1)[0])
        
        #We need to recover the original depth of image before augmentation
        #Since the augmentation process is based on normal image depth(8 bits, 0-255)
        x = image_gen.flow(data, batch_size = data.shape[0], shuffle = False)
        y = label
        
        #Dont forget to normalize the image back after done aumentation processing
        yield next(x), y
        

        
#Since the test dataset is small (only 400 pictures), so I decided to read all of them into RAM
test_x = []
for fn in test_fns:
    if ".jpg" in fn:
        test_x.append(np.asarray(load_target_img()(test_dir + fn)))

#dont forget to normalize the data
test_x = np.asarray(test_x, dtype = 'float64') / 255.0
test_x.shape

In [ ]:
#split the training dataset and validation dataset
train_dog_fns, valid_dog_fns = train_dog_fns[:1800], train_dog_fns[1800:]
train_cat_fns, valid_cat_fns = train_cat_fns[:1800], train_cat_fns[1800:]

In [ ]:
train_input_gen = image_generator(train_dir, train_dog_fns, train_cat_fns)
valid_input_gen = image_generator(train_dir, valid_dog_fns, valid_cat_fns)

train_image_gen = ImageDataGenerator(
            rotation_range = 40,
            shear_range = 0.2,
            width_shift_range = 0.2,
            height_shift_range = 0.2,
            zoom_range = 0.2,
            horizontal_flip = True,
            rescale = 1.0/255.0
            )

valid_image_gen = ImageDataGenerator( rescale = 1.0/255.0 )

train_aug_gen = image_augmentation_generator(train_input_gen, train_image_gen)
valid_aug_gen = image_augmentation_generator(valid_input_gen, valid_image_gen)

Let`s checkout some images

In [ ]:
#Use next to get one batch of data and label
batch_data, batch_label = next(valid_aug_gen)
print(f"batch data shape : {batch_data.shape}")
print(f"batch label shape : {batch_label.shape}")

plt.figure(figsize = (12,8))
nrow = 2
ncol = 4

for i in range(nrow*ncol):
    img = batch_data[i]
    label = batch_label[i]
    plt.subplot(nrow,ncol,i+1)
    plt.title(f"Label : {label}")
    plt.axis('off')
    plt.imshow(img)
plt.show()

In [ ]:
epochs = 200
batch_size = 32

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

weight_saving_path = "resnet50.hdf5"

#ModelCheckpoint will keep monitor your monitor item, and it will save the weight automatically
#when the monitor item is imporved
checkpoint = ModelCheckpoint(weight_saving_path, monitor = "val_loss", verbose = 1,
                            save_best_only = True, mode = "min")

#ReduceLROnPlateau will reduce your learning rate if the monitor item was not improve in patience times.
reduce_lr = ReduceLROnPlateau(monitor = "val_loss", verbose = 1, mode = "min", patience = 10,
                             factor = 0.5, min_lr = 1e-8)

#EarlyStopping will stop the training if the monitor item was not improve in patience times.
ES = EarlyStopping(monitor = "val_loss", patience = 50, mode = "min", verbose = 1)

training_callbacks = [checkpoint, ES, reduce_lr]

In [ ]:
history = model.fit_generator(train_aug_gen, epochs = epochs, verbose =1, validation_data = valid_aug_gen , validation_steps = (len(valid_dog_fns) + len(valid_cat_fns))//batch_size,
                                steps_per_epoch = (len(train_dog_fns)+len(train_cat_fns))//batch_size, callbacks = training_callbacks)

In [ ]:
model.load_weights('resnet50.hdf5')

In [ ]:
plt.figure(figsize = (8,6))
plt.plot(history.history['loss'], label = 'train_loss')
plt.plot(history.history['val_loss'], label = 'valid_loss')
plt.title("Loss of resnet50 on Dogs and Cats classification")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.figure(figsize = (8,6))
plt.plot(history.history['acc'], label = 'train_acc')
plt.plot(history.history['val_acc'], label = 'valid_acc')
plt.title("Accuracy of resnet50 on Dogs and Cats classification")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

Let's checkout some prediction result by image

In [ ]:
valid_data,valid_label =  next(valid_aug_gen)

def get_label_name(prediction):
    return "dog" if prediction < 0.5 else "cat"

plt.figure(figsize = (20,8))
nrow = 2
ncol = 5

for i,(img, label) in enumerate(zip(valid_data[:10], valid_label[:10])):
    
    predict_result = model.predict(np.expand_dims(img, axis = 0))[:,1]
    predict_result = get_label_name(predict_result[0])
    ground_truth = get_label_name(label[1])
    
    plt.subplot(nrow, ncol, i+1)
    plt.imshow(img)
    plt.title(f"predict : {predict_result}, ground truth : {ground_truth}")
    plt.axis("off")

plt.show()

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv')
submission['Predicted'] = model.predict(test_x)[:,1] 
submission.to_csv("v5.csv",header = ["ID", "Predicted"], index = False)